In [1]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

from sklearn import preprocessing, pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import IsolationForest

from cfmining.mip_algorithms import LinearRecourseActions, LinearRecourseActionsMulti
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy, MAPOCAM2
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier, GeneralClassifier, TreeClassifier, LinearClassifier, LinearRule, GeneralClassifier_Shap
from cfmining.action_set import ActionSet

from results import save_result

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

## Preparing dataset

In [3]:
data_dir = "data/"
data_name = 'german'
data_file = os.path.join(data_dir, '%s_processed.csv' % data_name)
## load and process data
german_df = pd.read_csv(data_file).reset_index(drop=True)
german_df = (german_df
             .assign(isMale=lambda df: (df['Gender']=='Male').astype(int))
             .drop(['PurposeOfLoan', 'Gender', 'OtherLoansAtStore'], axis=1)
            )

y = german_df['GoodCustomer']
# tranform y to [0, 1]
y = (y == 1).astype(int)
X = german_df.drop('GoodCustomer', axis=1)
X = X[['ForeignWorker', 'Single', 'Age', 'LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome', 'isMale']]

In [4]:
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=100)

In [5]:
outlier_detection = IsolationForest(contamination=0.1, n_estimators=50)
outlier_detection.fit(Xtr.values);
print(np.unique(outlier_detection.predict(Xtr.values), return_counts=True))

(array([-1,  1]), array([ 90, 810]))


## Setting ActionSet base parameters

In [6]:
action_set_base = ActionSet(X = X)
for feat in action_set_base:
    if feat.name in ['Age', 'JobClassIsSkilled', 'OwnsHouse', 'isMale', 'JobClassIsSkilled']:
        feat.mutable = False
        feat.step_direction = 1
    if feat.name in ['Single', 'ForeignWorker', 'RentsHouse']:
        feat.mutable = False
        feat.step_direction = -1
    if feat.name in ['LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome', 'NumberOfOtherLoansAtBank', 'MissedPayments',
                     'CriticalAccountOrLoansElsewhere', 'OtherLoansAtBank', 'Unemployed', 'YearsAtCurrentJob_lt_1']:
        feat.mutable = True
        feat.step_direction = -1
    if feat.name in ['YearsAtCurrentHome', 'NumberOfLiableIndividuals', 'HasTelephone', 'CheckingAccountBalance_geq_0',
                     'CheckingAccountBalance_geq_200', 'SavingsAccountBalance_geq_100', 'SavingsAccountBalance_geq_500',
                     'NoCurrentLoan', 'HasCoapplicant', 'HasGuarantor', 'YearsAtCurrentJob_geq_4']:
        feat.mutable = True
        feat.step_direction = 1
    
    feat.flip_direction = 1
    feat.update_grid()

## Logistic regression model

In [7]:
stand = preprocessing.StandardScaler()
clf_base = LogisticRegression(max_iter=1000, class_weight='balanced')
clf = pipeline.Pipeline([('std', stand), ('clf', clf_base)])

grid = GridSearchCV(
  clf, param_grid={'clf__C': np.logspace(-4, 3)},
  cv=5,
  scoring='roc_auc',
  verbose=0
)

grid.fit(Xtr, ytr)
clf_lgr = grid.best_estimator_

In [8]:
threshold=0.5
scores = pd.Series(clf_lgr.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < threshold].index

In [9]:
clf_lgr_mapocam = MonotoneClassifier(clf_lgr, X=Xtr, y=ytr, threshold=threshold)
coefficients = clf_lgr['clf'].coef_[0]/clf_lgr['std'].scale_
intercept = clf_lgr['clf'].intercept_[0]-coefficients@clf_lgr['std'].mean_

In [60]:
clf_lgr_shap_1 = GeneralClassifier_Shap(clf_lgr, X=Xtr, y=ytr, threshold=threshold)

X does not have valid feature names, but StandardScaler was fitted with feature names
  0%|          | 0/900 [00:00<?, ?it/s]

X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
  0%|          | 1/900 [00:00<01:45,  8.52it/s]X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
  0%|          | 3/900 [00:00<01:29, 10.07it/s]X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
  1%|          | 5/900 [00:00<01:19, 11.25it/s

In [11]:
#clf_lgr_shap = GeneralClassifier_Shap(clf_lgr, outlier_detection=outlier_detection, X=Xtr, y=ytr, threshold=threshold)

In [ ]:
clf_lgr_shap_2 = GeneralClassifier_Shap(clf_lgr, X=Xtr, y=ytr, threshold=threshold, method_predict_max="monotone",)

In [65]:
prob_max_1 = []
prob_max_2 = []
fixed_vars = [0, 1, 2]
for i in range(Xts.shape[0]):
    row = Xts.values[i, :]
    prob_max_1.append(clf_lgr_shap_1.predict_max(row, fixed_vars=fixed_vars))
    prob_max_2.append(clf_lgr_shap_2.predict_max(row, fixed_vars=fixed_vars))


prob [0.49465795]


  0%|          | 0/1 [00:00<?, ?it/s]

X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
100%|██████████| 1/1 [00:00<00:00,  8.31it/s]


[-0.00318288 -0.01321766 -0.00377574  0.02347338  0.01304599 -0.01404428
 -0.01772744]
[0.06045118 0.01220976 0.04847609 0.04510123 0.0225481  0.02650972
 0.00760493]
prob [0.49465795]
prob [0.54023921]


  0%|          | 0/1 [00:00<?, ?it/s]X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
100%|██████████| 1/1 [00:00<00:00,  8.61it/s]


...

[-0.00319497 -0.01321455 -0.0137638   0.0071274  -0.00286834 -0.0140373
 -0.01772203]
[0.06045118 0.01220976 0.04847609 0.04510123 0.0225481  0.02650972
 0.00760493]
prob [0.45241298]
prob [0.52601356]


  0%|          | 0/1 [00:00<?, ?it/s]X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
100%|██████████| 1/1 [00:00<00:00,  9.99it/s]


[-0.00316514 -0.01318637 -0.00375535  0.0235474   0.01889692 -0.01401078
  0.0076003 ]
[0.06045118 0.01220976 0.04847609 0.04510123 0.0225481  0.02650972
 0.00760493]
prob [0.52601356]
prob [0.51139402]


  0%|          | 0/1 [00:00<?, ?it/s]X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
100%|██████████| 1/1 [00:00<00:00, 12.87it/s]

[-0.0031735  -0.01320384 -0.00875758  0.02352616  0.00763116  0.01299431
 -0.01770928]
[0.06045118 0.01220976 0.04847609 0.04510123 0.0225481  0.02650972
 0.00760493]
prob [0.51139402]


In [70]:
prob_max_1 = np.array(prob_max_1)
prob_max_2 = np.array(prob_max_2)
np.median(prob_max_1 - prob_max_2)

0.0008851102611480699

In [ ]:
clf_lgr_shap = GeneralClassifier_Shap(clf_lgr, X=Xtr, y=ytr, threshold=threshold, method_predict_max="monotone",)
row = Xts.values[0, :]
row
fixed_vars = [0, 1, 2]
clf_lgr_shap.predict_max(row, fixed_vars)

In [59]:
row = Xts.values[0, :]
row
fixed_vars = [0, 1, 2]
clf_lgr_shap.predict_max(row, fixed_vars)

prob [0.49465795]


array([0.59139389])

## Preparing counterfactual parameters

In [26]:
action_set = copy.deepcopy(action_set_base)
action_set['Age'].mutable = False
action_set['Age'].update_grid()

action_set['LoanDuration'].step_type ="absolute"
action_set['LoanDuration'].step_size = 6
action_set['LoanDuration'].update_grid()

action_set['LoanAmount'].step_size = 0.1
action_set['LoanAmount'].update_grid()

print('ActionSet stats')
print('Number of actionable features:', sum([action.actionable for action in action_set]))
print('Mean number of actions per feature:', np.nanmean([len(action._grid) if action.actionable else np.nan for action in action_set]))
print('Max number of actions per feature:', np.nanmax([len(action._grid) if action.actionable else np.nan for action in action_set]))
log_combinations = int(np.prod([len(action._grid) for action in action_set if action_set.actionable]))
print('Number of combinations:', log_combinations)

ActionSet stats
Number of actionable features: 0
Mean number of actions per feature: nan
Max number of actions per feature: nan
Number of combinations: 172480


Mean of empty slice
All-NaN axis encountered


In [75]:
Xtr

,ForeignWorker,Single,Age,LoanDuration,LoanAmount,LoanRateAsPercentOfIncome,isMale
389,0,0,27,9,1449,3,0
621,0,1,32,18,1530,3,1
897,1,0,35,12,1592,3,0
702,0,0,47,24,1377,4,0
951,0,1,24,36,2145,2,1
...,...,...,...,...,...,...,...
452,0,1,34,12,2759,2,1
634,0,0,25,24,1355,3,0
626,0,1,41,6,2116,2,1
867,0,1,42,12,3331,2,1


In [79]:
[action._grid for action in action_set]

[array([0, 1]),
 array([0, 1]),
 array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]),
 array([ 6, 12, 18, 24, 30, 36, 42, 48, 54, 60]),
 array([  425,  1801,  3177,  4553,  5929,  7305,  8681, 10057, 11433,
        12809, 14185]),
 array([1, 2, 3, 4]),
 array([0, 1])]

In [13]:
percCalc = PercentileCalculator(action_set=action_set)

## Linear programming

In [ ]:
lp_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActions(action_set, individual, coefficients, intercept, threshold)
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_lgr_results[i] = {}
    lp_lgr_results[i]['enum'] = lp
    lp_lgr_results[i]['indiv'] = individual
    lp_lgr_results[i]['solution'] = np.array([int(s) if act.variable_type=='int' else float(s) for s, act in zip(lp.solution, action_set)])
    lp_lgr_results[i]['stats'] = lp.stats
    lp_lgr_results[i]['recourse'] = lp.recourse
    lp_lgr_results[i]['time'] = lp_time
    lp_lgr_results[i]['cost'] = criteria.f(lp_lgr_results[i]['solution']) if not any(np.isnan(lp_lgr_results[i]['solution'])) else float('inf')

## Greedy approach

In [ ]:
greedy_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    greedy = Greedy(action_set, individual, clf_lgr_mapocam, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_lgr_results[i] = {}
    greedy_lgr_results[i]['solution'] = greedy.solution
    greedy_lgr_results[i]['time'] = greedy_time
    greedy_lgr_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

## MAPOCAM

In [14]:
mapocam_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam,
                      max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_lgr_results[i] = {}
    mapocam_lgr_results[i]['enum'] = mapocam
    mapocam_lgr_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_lgr_results[i]['time'] = mapocam_time
    mapocam_lgr_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

  0%|          | 0/42 [00:00<?, ?it/s]

X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScale

## MAPOCAM2

In [98]:
mapocam2_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM2(
            action_set, 
            individual, 
            clf_lgr_mapocam, 
            outlier_detection,
            max_changes=float('inf'), 
            compare=criteria
    )
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam2_lgr_results[i] = {}
    mapocam2_lgr_results[i]['enum'] = mapocam
    mapocam2_lgr_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam2_lgr_results[i]['time'] = mapocam_time
    mapocam2_lgr_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')
    mapocam2_lgr_results[i]["prob_max_counter"] = mapocam.prob_max_counter
    mapocam2_lgr_results[i]["outlier_detection_counter"] = mapocam.outlier_detection_counter
    if i > 20:
        break

  0%|          | 0/40 [00:00<?, ?it/s]X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have 

In [99]:
prob_max_counter_1 = pd.DataFrame(mapocam2_lgr_results).T.prob_max_counter.values

In [100]:
prob_max_counter_1

array([0, 0, 0, 0, 0, 0, 0, 6, 0, 2, 0, 0], dtype=object)

In [39]:
mapocam2_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM2(
            action_set, 
            individual, 
            clf_lgr_shap, 
            outlier_detection,
            max_changes=float('inf'), 
            compare=criteria
    )
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam2_lgr_results[i] = {}
    mapocam2_lgr_results[i]['enum'] = mapocam
    mapocam2_lgr_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam2_lgr_results[i]['time'] = mapocam_time
    mapocam2_lgr_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')
    mapocam2_lgr_results[i]["prob_max_counter"] = mapocam.prob_max_counter
    mapocam2_lgr_results[i]["outlier_detection_counter"] = mapocam.outlier_detection_counter

  0%|          | 0/40 [00:00<?, ?it/s]X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
  2%|▎         | 1/40 [00:00<00:06,  6.22it/s]X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler 

[0.72924986]


X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
100%|██████████| 1/1 [00:00<00:00,  2.34it/s]
X does not have valid feature names, but StandardScaler was fitted with feature names


[0.71296911]


X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
100%|██████████| 1/1 [00:00<00:00,  2.36it/s]
X does not have valid feature names, but StandardScaler was fitted with feature names


...

[0.72924499]


X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
100%|██████████| 1/1 [00:00<00:00,  2.03it/s]
X does not have valid feature names, but StandardScaler was fitted with feature names


[0.71295175]


X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
100%|██████████| 1/1 [00:00<00:00,  2.33it/s]
X does not have valid feature names, but StandardScaler was fitted with feature names
100%|██████████| 40/40 [01:10<00:00,  1.77s/it]

[0.6967251]


In [40]:
prob_max_counter_2 = pd.DataFrame(mapocam2_lgr_results).T.prob_max_counter.values

In [41]:
prob_max_counter_2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object)

In [33]:
prob_max_counter_1.mean(), prob_max_counter_2.mean()

(1.4, 0.0)

## Performance comparison for single objective

In [ ]:
lp_lgr_costs = np.array([lp_lgr_results[i]['cost'] for i in denied_individuals])
greedy_lgr_costs = np.array([greedy_lgr_results[i]['cost'] for i in denied_individuals])
mapocam_lgr_costs = np.array([mapocam_lgr_results[i]['cost'] for i in denied_individuals])

lp_lgr_time = np.array([lp_lgr_results[i]['time'] for i in denied_individuals])
greedy_lgr_time = np.array([greedy_lgr_results[i]['time'] for i in denied_individuals])
mapocam_lgr_time = np.array([mapocam_lgr_results[i]['time'] for i in denied_individuals])

best_costs = np.min([lp_lgr_costs, greedy_lgr_costs, mapocam_lgr_costs], axis=0)

In [ ]:
lgr_dict = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_lgr_time)),
                'greedy': float(np.mean(greedy_lgr_time)),
                'mi-logistic': float(np.mean(lp_lgr_time)),
            },
            'Best solution rate':
            {
                'MAPOCAM':float(np.mean(mapocam_lgr_costs==best_costs)),
                'greedy': float(np.mean(greedy_lgr_costs==best_costs)),
                'mi-logistic': float(np.mean(lp_lgr_costs==best_costs)),
            },
            'Feasible rate':1-float(np.mean(np.isinf(mapocam_lgr_costs))),
            'Number of combinations':log_combinations
    
}
save_result(lgr_dict, 'german', 'logistic', 'MPC cost')

## Comparing Pareto enumeration

## Percentile vs changes

In [ ]:
mapocam_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=True, clean_suboptimal=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_pc[i] = {}
    mapocam_results_pc[i]['enum'] = mapocam
    mapocam_results_pc[i]['time'] = mapocam_time
    mapocam_results_pc[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    threshold, max_changes=3, enumeration_type='remove_number_actions',
                                    compare=PercentileChangesCriterion(individual, percCalc))
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_pc[i] = {}
    lp_results_pc[i]['enum'] = lp
    lp_results_pc[i]['time'] = lp_time
    lp_results_pc[i]['num'] = len(lp.solutions)

In [ ]:
bf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_pc[i] = {}
    bf_results_pc[i]['enum'] = bf
    bf_results_pc[i]['time'] = bf_time
    bf_results_pc[i]['num'] = len(bf.solutions)

In [ ]:
brute_sols_pc = np.array([bf_results_pc[i]['num'] for i in denied_individuals])
mapocam_sols_pc = np.array([mapocam_results_pc[i]['num'] for i in denied_individuals])
lp_sols_pc = np.array([lp_results_pc[i]['num'] for i in denied_individuals])
best_sols_pc = brute_sols_pc

brute_time_pc = np.array([bf_results_pc[i]['time'] for i in denied_individuals])
mapocam_time_pc = np.array([mapocam_results_pc[i]['time'] for i in denied_individuals])
lp_time_pc = np.array([lp_results_pc[i]['time'] for i in denied_individuals])

In [ ]:
lgr_dict_pc = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_time_pc)),
                'brute-force': float(np.mean(brute_time_pc)),
                'po-mi-logistic': float(np.mean(lp_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM':float(np.mean(mapocam_sols_pc==best_sols_pc)),
               'brute-force': float(np.mean(brute_sols_pc==best_sols_pc)),
               'po-mi-logistic': float(np.mean(lp_sols_pc==best_sols_pc)),
           }
    
}
save_result(lgr_dict_pc, 'german', 'logistic', 'MPC vs. #changes')

## Features

In [ ]:
mapocam_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam,
                      max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_feat[i] = {}
    mapocam_results_feat[i]['enum'] = mapocam
    mapocam_results_feat[i]['time'] = mapocam_time
    mapocam_results_feat[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    threshold, max_changes=3, enumeration_type='remove_dominated')
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_feat[i] = {}
    lp_results_feat[i]['enum'] = lp
    lp_results_feat[i]['time'] = lp_time
    lp_results_feat[i]['num'] = len(lp.solutions)

In [ ]:
bf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_feat[i] = {}
    bf_results_feat[i]['enum'] = bf
    bf_results_feat[i]['time'] = bf_time
    bf_results_feat[i]['num'] = len(bf.solutions)

In [ ]:
brute_sols_feat = np.array([bf_results_feat[i]['num'] for i in denied_individuals])
mapocam_sols_feat = np.array([mapocam_results_feat[i]['num'] for i in denied_individuals])
lp_sols_feat = np.array([lp_results_feat[i]['num'] for i in denied_individuals])
best_sols_feat = brute_sols_feat

brute_time_feat = np.array([bf_results_feat[i]['time'] for i in denied_individuals])
mapocam_time_feat = np.array([mapocam_results_feat[i]['time'] for i in denied_individuals])
lp_time_feat = np.array([lp_results_feat[i]['time'] for i in denied_individuals])

In [ ]:
lgr_dict_feat = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_time_feat)),
                'brute-force': float(np.mean(brute_time_feat)),
                'po-mi-logistic': float(np.mean(lp_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM':float(np.mean(mapocam_sols_feat==best_sols_feat)),
               'brute-force': float(np.mean(brute_sols_feat==best_sols_feat)),
               'po-mi-logistic': float(np.mean(lp_sols_feat==best_sols_feat)),
           }
    
}
save_result(lgr_dict_feat, 'german', 'logistic', 'feature')

## Random forest

In [45]:
from sklearn import tree, ensemble
from cfmining.mip_algorithms import ForestRecourseActions
from cfmining.predictors import TreeClassifier

In [46]:
clf_rt = ensemble.RandomForestClassifier(n_estimators=5, max_depth=5, max_leaf_nodes=31,
                                         class_weight='balanced_subsample')
clf_rt.fit(Xtr, ytr);

## Preparing counterfactual parameters

In [47]:
action_set_tree = copy.deepcopy(action_set_base)
action_set_tree.embed_forest(clf_rt)
for action in action_set_tree:
    action.flip_direction = action.step_direction
    
print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_tree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_tree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_tree]))
rt_combinations = int(np.prod([len(action._grid) for action in action_set_tree if action.actionable]))
print('Number of combinations:', rt_combinations)

ActionSet stats
Number of actionable features: 3
Mean number of actions per feature: 9.285714285714286
Max number of actions per feature: 27
Number of combinations: 1904


In [48]:
scores = pd.Series(clf_rt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < threshold].index

In [49]:
percCalc = PercentileCalculator(action_set=action_set_tree)

In [50]:
mapocam_clf_rf = TreeClassifier(clf_rt, Xtr, ytr, threshold=threshold)
mapocam_clf_rf_fast = TreeClassifier(clf_rt, Xtr, ytr, threshold=threshold, use_predict_max=True)

In [51]:
clf_rf_shap = GeneralClassifier_Shap(clf_rt, X=Xtr, y=ytr, threshold=threshold)

In [52]:
imp = list(zip(Xtr.columns, clf_rf_shap.importances, clf_rf_shap.shap_max))
imp.sort(key=lambda x: x[1], reverse=True)
imp

[('LoanDuration', 0.09061631274461034, 0.243646646062386),
 ('LoanAmount', 0.036798365851921644, 0.07029098510744773),
 ('Age', 0.03161958626726464, 0.13768185864239407),
 ('Single', 0.02250859137918783, 0.0648081455533384),
 ('ForeignWorker', 0.015064389479717867, 0.18179992504239437),
 ('LoanRateAsPercentOfIncome', 0.01274178859934849, 0.12403375808021526),
 ('isMale', 0.010102566725003013, 0.0639169046402256)]

In [49]:
clf_rf_shap = GeneralClassifier_Shap(clf_rt, outlier_detection=outlier_detection, X=Xtr, y=ytr, threshold=threshold)

ExactExplainer explainer: 901it [00:13, 25.78it/s]                          


In [50]:
imp = list(zip(Xtr.columns, clf_rf_shap.importances, clf_rf_shap.shap_max))
imp.sort(key=lambda x: x[1], reverse=True)
imp

[('LoanDuration', 0.08396093310336354, 0.2779718140399186),
 ('Age', 0.057508943987659815, 0.19671331484564517),
 ('LoanAmount', 0.03662699766295823, 0.13678909653787466),
 ('Single', 0.021008334734129636, 0.06347488540425911),
 ('isMale', 0.012080998224669705, 0.08227360496431882),
 ('LoanRateAsPercentOfIncome', 0.01144972571009471, 0.12564793716935008),
 ('ForeignWorker', 0.006460803714000895, 5.947448460845439e-05)]

## Linear programming

In [ ]:
lp_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual,
                               clf_rt, threshold=threshold,
                               max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_rf_results[i] = {}
    lp_rf_results[i]['enum'] = lp
    lp_rf_results[i]['indiv'] = individual
    lp_rf_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_rf_results[i]['time'] = lp_time
    lp_rf_results[i]['cost'] = criteria.f(lp_rf_results[i]['solution']) if not any(np.isnan(lp_rf_results[i]['solution'])) else float('inf')

## Greedy approach

In [ ]:
greedy_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    greedy = Greedy(action_set_tree, individual, mapocam_clf_rf_fast, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_rf_results[i] = {}
    greedy_rf_results[i]['solution'] = greedy.solution
    greedy_rf_results[i]['time'] = greedy_time
    greedy_rf_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

## MAPOCAM

In [61]:
mapocam_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf, max_changes=float('inf'),
                      compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results[i] = {}
    mapocam_rf_results[i]['enum'] = mapocam
    mapocam_rf_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results[i]['time'] = mapocam_time
    mapocam_rf_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

 39%|███▉      | 15/38 [00:00<00:00, 144.54it/s]

100%|██████████| 38/38 [00:00<00:00, 125.31it/s]


In [ ]:
mapocam_rf_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf_fast, max_changes=float('inf'),
                      compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_plus[i] = {}
    mapocam_rf_results_plus[i]['enum'] = mapocam
    mapocam_rf_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results_plus[i]['time'] = mapocam_time
    mapocam_rf_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

## Mapocam 2

In [82]:
mapocam2_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM2(
        action_set, 
        individual, 
        clf_rf_shap, 
        outlier_detection,
        max_changes=float('inf'), 
        compare=criteria
    )
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam2_rf_results[i] = {}
    mapocam2_rf_results[i]['enum'] = mapocam
    mapocam2_rf_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam2_rf_results[i]['time'] = mapocam_time
    mapocam2_rf_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')
    mapocam2_rf_results[i]["outlier_detection_counter"] = mapocam.outlier_detection_counter

  0%|          | 0/38 [00:00<?, ?it/s]X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  0%|          | 0/38 [00:00<?, ?it/s]


AttributeError: 'IsolationForest' object has no attribute 'predict_proba'

In [81]:
mapocam2_rf_results

{0: {'enum': <cfmining.algorithms.MAPOCAM2 at 0x7fc542178fd0>,
  'solution': None,
  'time': 0.5197362200001407,
  'cost': inf,
  'outlier_detection_counter': 3},
 1: {'enum': <cfmining.algorithms.MAPOCAM2 at 0x7fc542305180>,
  'solution': None,
  'time': 0.027307844999995723,
  'cost': inf,
  'outlier_detection_counter': 1},
 6: {'enum': <cfmining.algorithms.MAPOCAM2 at 0x7fc542304880>,
  'solution': None,
  'time': 0.11601910100034729,
  'cost': inf,
  'outlier_detection_counter': 4},
 7: {'enum': <cfmining.algorithms.MAPOCAM2 at 0x7fc54227b4c0>,
  'solution': None,
  'time': 0.02056248200005939,
  'cost': inf,
  'outlier_detection_counter': 1},
 8: {'enum': <cfmining.algorithms.MAPOCAM2 at 0x7fc54217b4c0>,
  'solution': None,
  'time': 0.026366496000264306,
  'cost': inf,
  'outlier_detection_counter': 1},
 9: {'enum': <cfmining.algorithms.MAPOCAM2 at 0x7fc542178040>,
  'solution': None,
  'time': 0.018908365000243066,
  'cost': inf,
  'outlier_detection_counter': 1},
 11: {'enum': 

In [64]:
mapocam.prob_max_counter, mapocam.outlier_detection_counter

(0, 0)

In [83]:
np.sum([v["time"] for v in mapocam_rf_results.values() if v["solution"] is not None]), np.sum([v["time"] for v in mapocam2_rf_results.values() if v["solution"] is not None])

(0.6279486449911929, 1.5361950600035925)

## Performance comparison for single objective

In [ ]:
lp_rf_costs = np.array([lp_rf_results[i]['cost'] for i in denied_individuals])
greedy_rf_costs = np.array([greedy_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs = np.array([mapocam_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs_plus = np.array([mapocam_rf_results_plus[i]['cost'] for i in denied_individuals])
best_costs_rf = np.min([lp_rf_costs, greedy_rf_costs, mapocam_rf_costs, mapocam_rf_costs_plus], axis=0)

lp_rf_time = np.array([lp_rf_results[i]['time'] for i in denied_individuals])
greedy_rf_time = np.array([greedy_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time = np.array([mapocam_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time_plus = np.array([mapocam_rf_results_plus[i]['time'] for i in denied_individuals])

In [ ]:
tree_dict = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time)),
                'greedy': float(np.mean(greedy_rf_time)),
                'mi-trees': float(np.mean(lp_rf_time)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_costs_plus==best_costs_rf)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_costs==best_costs_rf)),
               'greedy': float(np.mean(greedy_rf_costs==best_costs_rf)),
               'mi-trees': float(np.mean(lp_rf_costs==best_costs_rf)),
           },
             'Feasible rate':1-float(np.mean(np.isinf(mapocam_rf_costs_plus))),
             'Number of combinations': rt_combinations
    
}
save_result(tree_dict, 'german', 'tree', 'MPC cost')

## Comparing Pareto enumeration

## Percentile vs changes

In [ ]:
mapocam_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc[i] = {}
    mapocam_rf_results_pc[i]['enum'] = mapocam
    mapocam_rf_results_pc[i]['time'] = mapocam_time
    mapocam_rf_results_pc[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_rf_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf_fast, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc_plus[i] = {}
    mapocam_rf_results_pc_plus[i]['enum'] = mapocam
    mapocam_rf_results_pc_plus[i]['time'] = mapocam_time
    mapocam_rf_results_pc_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear', compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_pc[i] = {}
    lp_rf_results_pc[i]['enum'] = lp
    lp_rf_results_pc[i]['time'] = lp_time
    lp_rf_results_pc[i]['num'] = len(lp.solutions)

In [ ]:
bf_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_pc[i] = {}
    bf_rf_results_pc[i]['enum'] = bf
    bf_rf_results_pc[i]['time'] = bf_time
    bf_rf_results_pc[i]['num'] = len(bf.solutions)

In [ ]:
lp_rf_sols_pc = np.array([lp_rf_results_pc[i]['num'] for i in denied_individuals])
brute_rf_sols_pc = np.array([bf_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc = np.array([mapocam_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['num'] for i in denied_individuals])
best_rf_sols_pc = brute_rf_sols_pc

lp_rf_time_pc = np.array([lp_rf_results_pc[i]['time'] for i in denied_individuals])
brute_rf_time_pc = np.array([bf_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc = np.array([mapocam_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['time'] for i in denied_individuals])

In [ ]:
tree_dict_pc = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_pc_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time_pc)),
                'brute-force': float(np.mean(brute_rf_time_pc)),
                'po-mi-trees': float(np.mean(lp_rf_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_sols_pc_plus==best_rf_sols_pc)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_sols_pc==best_rf_sols_pc)),
               'brute-force': float(np.mean(brute_rf_sols_pc==best_rf_sols_pc)),
               'po-mi-trees': float(np.mean(lp_rf_sols_pc==best_rf_sols_pc)),
           }
    
}
save_result(tree_dict_pc, 'german', 'tree', 'MPC vs. #changes')

## Features

In [ ]:
mapocam_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat[i] = {}
    mapocam_rf_results_feat[i]['enum'] = mapocam
    mapocam_rf_results_feat[i]['time'] = mapocam_time
    mapocam_rf_results_feat[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_rf_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat_plus[i] = {}
    mapocam_rf_results_feat_plus[i]['enum'] = mapocam
    mapocam_rf_results_feat_plus[i]['time'] = mapocam_time
    mapocam_rf_results_feat_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_feat[i] = {}
    lp_rf_results_feat[i]['enum'] = lp
    lp_rf_results_feat[i]['time'] = lp_time
    lp_rf_results_feat[i]['num'] = len(lp.solutions)

In [ ]:
bf_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_feat[i] = {}
    bf_rf_results_feat[i]['enum'] = bf
    bf_rf_results_feat[i]['time'] = bf_time
    bf_rf_results_feat[i]['num'] = len(bf.solutions)

In [ ]:
brute_rf_sols_feat = np.array([bf_rf_results_feat[i]['num'] for i in denied_individuals])
lp_rf_sols_feat = np.array([lp_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat = np.array([mapocam_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['num'] for i in denied_individuals])
best_rf_sols_feat = brute_rf_sols_feat

brute_rf_time_feat = np.array([bf_rf_results_feat[i]['time'] for i in denied_individuals])
lp_rf_time_feat = np.array([lp_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat = np.array([mapocam_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['time'] for i in denied_individuals])

In [ ]:
tree_dict_feat = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_feat_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time_feat)),
                'brute-force': float(np.mean(brute_rf_time_feat)),
                'po-mi-trees': float(np.mean(lp_rf_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_sols_feat_plus==best_rf_sols_feat)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_sols_feat==best_rf_sols_feat)),
               'brute-force': float(np.mean(brute_rf_sols_feat==best_rf_sols_feat)),
               'po-mi-trees': float(np.mean(lp_rf_sols_feat==best_rf_sols_feat)),
           }
    
}
save_result(tree_dict_feat, 'german', 'tree', 'feature')

## Monotone trees

In [ ]:
from sklearn import tree, ensemble
import lightgbm
from cfmining.predictors import MonotoneTree

In [ ]:
monotone_constraints = [action.step_direction for action in action_set_base]
clf_mt = lightgbm.LGBMClassifier(n_estimators=10, max_depth=10, num_leaves=63,
                                 monotone_constraints=monotone_constraints, class_weight='balanced')
clf_mt.fit(Xtr, ytr)

## Preparing counterfactual parameters

In [ ]:
action_set_mtree = copy.deepcopy(action_set_base)
action_set_mtree.embed_forest(clf_mt)
for action in action_set_mtree:
    action.flip_direction = action.step_direction

print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_mtree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_mtree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_mtree]))
mt_combinations = int(np.prod([len(action._grid) for action in action_set_mtree if action.actionable]))
print('Number of combinations:', mt_combinations)

In [ ]:
scores = pd.Series(clf_mt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < threshold].index

In [ ]:
percCalc = PercentileCalculator(action_set=action_set_mtree)

In [ ]:
mapocam_clf_mt = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold)
mapocam_clf_mt_fast = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold, use_predict_max=True)
mapocam_clf_mt_mon = MonotoneTree(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold, use_predict_max=True)

## Linear programming

In [ ]:
lp_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=threshold, max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_mt_results[i] = {}
    lp_mt_results[i]['enum'] = lp
    lp_mt_results[i]['indiv'] = individual
    lp_mt_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_mt_results[i]['time'] = lp_time
    lp_mt_results[i]['cost'] = criteria.f(lp_mt_results[i]['solution']) if not any(np.isnan(lp_mt_results[i]['solution'])) else float('inf')

## Greedy approach

In [ ]:
greedy_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    greedy = Greedy(action_set_mtree, individual, mapocam_clf_mt_fast, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_mt_results[i] = {}
    greedy_mt_results[i]['solution'] = greedy.solution
    greedy_mt_results[i]['time'] = greedy_time
    greedy_mt_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

## MAPOCAM

In [ ]:
mapocam_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, 
                      mapocam_clf_mt, max_changes=float('inf'), 
                      compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results[i] = {}
    mapocam_mt_results[i]['enum'] = mapocam
    mapocam_mt_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results[i]['time'] = mapocam_time
    mapocam_mt_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

In [ ]:
mapocam_mt_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, 
                      mapocam_clf_mt_fast, max_changes=float('inf'),
                      compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_plus[i] = {}
    mapocam_mt_results_plus[i]['enum'] = mapocam
    mapocam_mt_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_plus[i]['time'] = mapocam_time
    mapocam_mt_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

In [ ]:
mapocam_mt_results_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_mon[i] = {}
    mapocam_mt_results_mon[i]['enum'] = mapocam
    mapocam_mt_results_mon[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_mon[i]['time'] = mapocam_time
    mapocam_mt_results_mon[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

In [ ]:
lp_mt_costs = np.array([lp_mt_results[i]['cost'] for i in denied_individuals])
greedy_mt_costs = np.array([greedy_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs = np.array([mapocam_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_plus = np.array([mapocam_mt_results_plus[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_mon = np.array([mapocam_mt_results_mon[i]['cost'] for i in denied_individuals])
best_costs_mt_rf = np.min([lp_mt_costs, greedy_mt_costs, mapocam_mt_costs, mapocam_mt_costs_plus, mapocam_mt_costs_mon], axis=0)

lp_mt_time = np.array([lp_mt_results[i]['time'] for i in denied_individuals])
greedy_mt_time = np.array([greedy_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time = np.array([mapocam_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time_plus = np.array([mapocam_mt_results_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_mon = np.array([mapocam_mt_results_mon[i]['time'] for i in denied_individuals])

In [ ]:
tree_dict = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time)),
                'greedy': float(np.mean(greedy_mt_time)),
                'mi-trees': float(np.mean(lp_mt_time)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_costs_mon==best_costs_mt_rf)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_costs_plus==best_costs_mt_rf)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_costs==best_costs_mt_rf)),
               'greedy': float(np.mean(greedy_mt_costs==best_costs_mt_rf)),
               'mi-trees': float(np.mean(lp_mt_costs==best_costs_mt_rf)),
           },
             'Feasible rate':1-float(np.mean(np.isinf(mapocam_mt_costs_mon))),
             'Number of combinations': mt_combinations
    
}
save_result(tree_dict, 'german', 'monotone-tree', 'MPC cost')

## Comparing Pareto enumeration

## Percentile vs changes

In [ ]:
mapocam_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, 
                      mapocam_clf_mt, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc[i] = {}
    mapocam_mt_results_pc[i]['enum'] = mapocam
    mapocam_mt_results_pc[i]['time'] = mapocam_time
    mapocam_mt_results_pc[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual,
                      mapocam_clf_mt_fast, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_plus[i] = {}
    mapocam_mt_results_pc_plus[i]['enum'] = mapocam
    mapocam_mt_results_pc_plus[i]['time'] = mapocam_time
    mapocam_mt_results_pc_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_pc_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual,
                      mapocam_clf_mt_mon, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_mon[i] = {}
    mapocam_mt_results_pc_mon[i]['enum'] = mapocam
    mapocam_mt_results_pc_mon[i]['time'] = mapocam_time
    mapocam_mt_results_pc_mon[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear', compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_pc[i] = {}
    lp_mt_results_pc[i]['enum'] = lp
    lp_mt_results_pc[i]['time'] = lp_time
    lp_mt_results_pc[i]['num'] = len(lp.solutions)

In [ ]:
bf_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_pc[i] = {}
    bf_mt_results_pc[i]['enum'] = bf
    bf_mt_results_pc[i]['time'] = bf_time
    bf_mt_results_pc[i]['num'] = len(bf.solutions)

In [ ]:
brute_mt_sols_pc = np.array([bf_mt_results_pc[i]['num'] for i in denied_individuals])
lp_mt_sols_pc = np.array([lp_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc = np.array([mapocam_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['num'] for i in denied_individuals])
best_mt_sols_pc = brute_mt_sols_pc

brute_mt_time_pc = np.array([bf_mt_results_pc[i]['time'] for i in denied_individuals])
lp_mt_time_pc = np.array([lp_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc = np.array([mapocam_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['time'] for i in denied_individuals])

In [ ]:
mon_tree_dict_pc = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_pc_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_pc_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time_pc)),
                'brute-force': float(np.mean(brute_mt_time_pc)),
                'po-mi-trees': float(np.mean(lp_mt_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_sols_pc_plus==mapocam_mt_sols_pc_mon)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_sols_pc_plus==best_mt_sols_pc)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_sols_pc==best_mt_sols_pc)),
               'brute-force': float(np.mean(brute_mt_sols_pc==best_mt_sols_pc)),
               'po-mi-trees': float(np.mean(lp_mt_sols_pc==best_mt_sols_pc)),
           }
    
}
save_result(mon_tree_dict_pc, 'german', 'monotone-tree', 'MPC vs. #changes')

## Features

In [ ]:
mapocam_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat[i] = {}
    mapocam_mt_results_feat[i]['enum'] = mapocam
    mapocam_mt_results_feat[i]['time'] = mapocam_time
    mapocam_mt_results_feat[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_plus[i] = {}
    mapocam_mt_results_feat_plus[i]['enum'] = mapocam
    mapocam_mt_results_feat_plus[i]['time'] = mapocam_time
    mapocam_mt_results_feat_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_feat_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_mon[i] = {}
    mapocam_mt_results_feat_mon[i]['enum'] = mapocam
    mapocam_mt_results_feat_mon[i]['time'] = mapocam_time
    mapocam_mt_results_feat_mon[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_feat[i] = {}
    lp_mt_results_feat[i]['enum'] = lp
    lp_mt_results_feat[i]['time'] = lp_time
    lp_mt_results_feat[i]['num'] = len(lp.solutions)

In [ ]:
bf_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_feat[i] = {}
    bf_mt_results_feat[i]['enum'] = bf
    bf_mt_results_feat[i]['time'] = bf_time
    bf_mt_results_feat[i]['num'] = len(bf.solutions)

In [ ]:
brute_mt_sols_feat = np.array([bf_mt_results_feat[i]['num'] for i in denied_individuals])
lp_mt_sols_feat = np.array([lp_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat = np.array([mapocam_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['num'] for i in denied_individuals])
best_mt_sols_feat = brute_mt_sols_feat

brute_mt_time_feat = np.array([bf_mt_results_feat[i]['time'] for i in denied_individuals])
lp_mt_time_feat = np.array([lp_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat = np.array([mapocam_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['time'] for i in denied_individuals])

In [ ]:
print('Feasible rate:', np.mean(best_mt_sols_feat!=0))

In [ ]:
mon_tree_dict_feat = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_feat_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_feat_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time_feat)),
                'brute-force': float(np.mean(brute_mt_time_feat)),
                'po-mi-trees': float(np.mean(lp_mt_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_sols_feat_mon==best_mt_sols_feat)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_sols_feat_plus==best_mt_sols_feat)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_sols_feat==best_mt_sols_feat)),
               'brute-force': float(np.mean(brute_mt_sols_feat==best_mt_sols_feat)),
               'po-mi-trees': float(np.mean(lp_mt_sols_feat==best_mt_sols_feat)),
           }
    
}
save_result(mon_tree_dict_feat, 'german', 'monotone-tree', 'feature')